# (1)

---
## Вопросы ДЗ
- А точно нужно сортировать по вероятности?
    - зависит от приоритетов: если мы хотим просто привлечь клиента, то по вероятности / популярности товара; если мы хотим максимизировать прибыль, то лучше сортировать по вероятности, умноженной на прибыльность товара; возможны и другие варианты - продать какие-то конкретные товары; но лучше всего комбинировать эти подходы;
- Какую метрику использовать?
    - Money Precision@10 и/или просто Precision@10 в зависимости от целей;
- Сколько раз в неделю отпрпавляем рассылку?
    - 1 раз в неделю (сужу по себе), чтобы не быть назойливыми;
- В какое время отправляем рассылку?
    - Лучше накануне самых нагруженных дней (если это еда, то наверное лучше делать рассылку в пятницу перед выходными в утреннее время - вечером в пятницу всем не до этого, а днем все работают хотя лучше провести A/B тестирование), а так надо опираться на собственную статистику магазина;
- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?
    - в разные дни можно давать разные рекомендации (например иходить из загруженности склада того или иного товара - добавить в качестве веса); можно брать товары из разных топов: самые покупаемые, самые понравившиеся, самые комментируемые и тд;
- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?
    - можно отсекать веса в зависимости от типа товара (например ввести ограничение - рекомендовать 1 товар из смежной категории), если исходные данные не промаркированы типами товаров - можно применить ML (к примеру, с использованием tfidf - хотя в магазине всегда все продается по категориям, поэтому это должно уже быть)
---

# (2)
- Доделать прошлые домашния задания.
    - доделаны
---

# (3)
- Почитать статьи из материалов к вебинару (все полезные!):
https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe
https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-2-alternating-least-square-als-matrix-4a76c58714a1
https://towardsdatascience.com/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b
https://bond-kirill-alexandrovich.medium.com/%D1%84%D0%B0%D0%BA%D1%82%D0%BE%D1%80%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F-%D0%B2-%D1%80%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D1%85-%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%B0%D1%85-%D1%87%D1%82%D0%BE-%D1%82%D0%B0%D0%BA%D0%BE%D0%B5-%D0%B8-%D1%81-%D1%87%D0%B5%D0%BC-%D0%B5%D1%91-%D0%B5%D0%B4%D1%8F%D1%82-477ba002df8
    - часть прочитана полностью, часть -- бегло
---

# (4)
- Поиграться с параметрами ALS (GridSearch, RandomSearch)

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
import os, sys, itertools
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
path = r'../_ADDS/webinar_3/data/transaction_data.csv'

In [34]:
data = pd.read_csv(path)
data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [35]:
data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [36]:
data_train['price'] = data_train['sales_value'] / (np.maximum(data_train['quantity'], 1))
data_train['price'].max()

/home/arthur/.conda/envs/implicit_39/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


499.99

In [11]:
# < 1$
data_train['price'].quantile(0.20)

0.99

In [12]:
# > 100$
data_train['price'].quantile(0.99995)

84.8129592499882

In [38]:
data_train

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,1.39
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0,0.82
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0,0.99
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0,1.21
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,1.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2488946,98,41944918665,684,945779,2,2.00,421,0.00,1705,98,0.0,0.0,1.00
2488947,98,41944918665,684,993617,2,2.00,421,0.00,1705,98,0.0,0.0,1.00
2488948,98,41944918665,684,1128647,2,2.00,421,0.00,1705,98,0.0,0.0,1.00
2488949,98,41944918665,684,9526886,2,0.60,421,0.00,1705,98,0.0,0.0,0.30


----

## Домашнее задание

**1. Попытаться ответить на вопросы / выдвинуть гипотезы**

**2. Доделать прошлые домашки!**

**3. Почитать статьи из материалов к вебинару (все полезные!)**

**4. Поиграться с параметрами ALS (GridSearch, RandomSearch)**

preprocessing-part

In [42]:
def prefilter_items(data, irrelevant=None, price_thresholds=(0, 1000000000)):
    """
    irrelevant - list of irrelevant departments of products to remove from data
    price_thresholds[0] - threshold of cheap items (min price)
    price_thresholds[1] - threshold of expensive items (max price)
    """
    # Уберем самые популярные товары (их и так купят)
    popularity = data.groupby('item_id')['user_id'].nunique().reset_index() / data['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    top_popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]
    
    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    max_day = data['day'].max()
    items_365 = data.loc[(data['day'] <= max_day) & (data['day'] >= max_day-365), 'item_id'].unique().tolist()
    data = data.loc[data['item_id'].isin(items_365)]
    
    # Уберем не интересные для рекоммендаций категории (department)
    if irrelevant:
        products = pd.DataFrame(r'../_ADDS/webinar_3/data/product.csv')
        relevant_products = products.loc[~products['DEPARTMENT'].isin(irrelevant), 'PRODUCT_ID'].inique().tolist()
        data = data.loc[data['item_id'].isin(relevant_products)]
    
    # Уберем слишком дешевые / дорогие товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    if price_thresholds[0] or price_thresholds[1] != 1000000000:
        data = data.loc[(data['price'] >= price_thresholds[0]) & (data['price'] <= price_thresholds[1])]
    
    # ...
    return data
    
def postfilter_items(user_id, recommednations):
    pass

In [44]:
data_train = prefilter_items(data_train)
data_test = prefilter_items(data_train)

In [47]:
data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,1.39
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0,0.82


In [50]:
popularity = data_train.groupby('item_id')['sales_value'].sum().reset_index()
popularity.rename(columns={'sales_value': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [52]:
# Заведем фиктивный item_id
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

In [53]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='sales_value',
                                  aggfunc='count', 
                                  fill_value=0)

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [54]:
# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

In [56]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

ALS-part

In [74]:
def get_recommendations(user, model, N=5):
    return [id_to_itemid[rec[0]] for rec in model.recommend(userid=userid_to_id[user], 
                                                            user_items=sparse_user_item,
                                                            N=N, 
                                                            filter_already_liked_items=False, 
                                                            filter_items=None, 
                                                            recalculate_user=True)]

def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    return precision

def als_grid_search(user_item_matrix, rec_results, params, N=5):
    scores = []
    for factors, reg, iters in itertools.product(*params.values()):
        model = AlternatingLeastSquares(factors=factors, 
                                        regularization=reg,
                                        iterations=iters, 
                                        calculate_training_loss=True, 
                                        num_threads=6)
        model.fit(csr_matrix(user_item_matrix).T, show_progress=False)
        
        rec_results['als'] = rec_results['user_id'].apply(lambda x: get_recommendations(x,model=model,N=N))
        precision = rec_results.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
        score = {'factors': factors,
                 'regularization': reg,
                 'iterations': iters,
                 'precision': precision}
        scores.append(score)
    return scores

def best_params(scores, all=False):
    df = pd.DataFrame(scores, index=range(len(scores))).sort_values('precision', ascending=False)
    if all:
        return df
    df = df.head(1)
    return df.to_dict()

In [59]:
rec_results = data_test.groupby('user_id')['item_id'].unique().reset_index()
rec_results.columns=['user_id', 'actual']
rec_results.head(2)

,user_id,actual
0,1,"[825123, 840361, 845307, 852014, 856942, 91267..."
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55..."


In [70]:
%%time
parameters = {'factors': [32, 64, 128],
              'regularization': [0.0001, 0.001, 0.01],
              'iterations': [10, 15, 20]}

scores = als_grid_search(user_item_matrix, rec_results, parameters, N=5)

CPU times: user 1h 1min 40s, sys: 46min 16s, total: 1h 47min 56s
Wall time: 14min 59s


In [75]:
best_params(scores, True)

,factors,regularization,iterations,precision
20,128,0.0001,20,0.79376
23,128,0.0010,20,0.79040
19,128,0.0001,15,0.79000
22,128,0.0010,15,0.78512
26,128,0.0100,20,0.78424
25,128,0.0100,15,0.78424
21,128,0.0010,10,0.77776
18,128,0.0001,10,0.77456
24,128,0.0100,10,0.77080
14,64,0.0010,20,0.67872


In [76]:
print('Наилучшие показатели продемонстрировала модель со следующими параметрами:')
best_params(scores)

Наилучшие показатели продемонстрировала модель со следующими параметрами:


{'factors': {20: 128},
 'regularization': {20: 0.0001},
 'iterations': {20: 20},
 'precision': {20: 0.7937599999999999}}